In [ ]:
!pip install -r requirements.txt
!pip install rouge python-box

In [2]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.bart.finetune_model import load_bart
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu
from src.model.bart import BART, BART_finetune
from src.loaders.finetune_loader import FineTuneLoader
import pytorch_lightning as pl
from box import Box
from collections import namedtuple
from src.utils.load_data import RIA
from src.utils.tokenizer import CustomTokenizer
from nltk.tokenize import sent_tokenize
from transformers import BartConfig
from pytorch_lightning import Trainer
from tqdm import tqdm
from rouge import Rouge
import pandas as pd
from src.utils import load_data
import numpy as np
import torch
import os

In [3]:
pl.seed_everything(2022)

Global seed set to 2022


2022

### Two news datasets: 'Газета', 'РИА Новости'.

"РИА Новости" gives only text and headlines, "Газета" consists of text, summary, headlines.

Preprocess data for both datasets

In [4]:
TEST_SIZE = 0.02
N_ROWS = 500_000
CHUNK_SIZE = 50_000

In [ ]:
load_data.collect_gazeta()
RIA('data/ria.json.gz', N_ROWS, CHUNK_SIZE).get_data()

499999it [27:06, 307.41it/s]


Load data for both datasets

In [5]:
data_ria = RIA.load_data('data/ria')
data_ria['text'] = data_ria['text'].apply(lambda x: x.strip(' '))
data_ria['title'] = data_ria['title'].apply(lambda x: x.strip(' '))
data_ria_train, data_ria_val = train_test_split(data_ria, test_size = TEST_SIZE)
data_gazeta_train = pd.read_csv('data/gazeta/gazeta_train.csv')
data_gazeta_val = pd.read_csv('data/gazeta/gazeta_val.csv')

In [39]:
data_ria_train['text'].iloc[0]

'глава мид рф сергей лавров по инициативе американской стороны обсудил в ходе телефонного разговора в четверг с государственным секретарем сша хиллари клинтон проект новой резолюции по ирану, а также перспективы реализации схемы обмена иранского низкообогащенного (3,5%) урана на обогащенный до 20% для тегеранского исследовательского реактора, сообщает в пятницу департамент информации и печати мид рф. в настоящее время представители стран "шестерки" посредников по урегулированию ситуации вокруг ядерной программы ирана, в которую входят рф, сша, кнр, франция, великобритания и германия, обсуждают проект резолюции о возможных дополнительных санкциях против ирана. ранее "шестерка" не могла договориться по американским предложениям, однако 18 мая клинтон объявила, что рф, сша и кнр достигли консенсуса по новой резолюции, разработанного американской стороной. "состоялся обмен мнениями о ситуации вокруг иранской ядерной программы в контексте работы в совете безопасности оон над проектом резолю

In [ ]:
data_ria.head()

,text,title
0,"большая часть из 33 детей, которых граждане с...","большинство детей, которых пытались увезти в с..."
1,"премьер-министр украины, кандидат в президент...","луценко будет работать в команде тимошенко, ес..."
2,"до 7 февраля - того дня, когда граждане украи...","""лютые"" выборы: есть ли вероятность второго ма..."
3,группа вооруженных людей в ночь с субботы на ...,жертвами бойни на севере мексики стали 13 моло...
4,немецкий теннисист михаэль беррер стал победи...,немец беррер выиграл теннисный турнир на родин...


In [ ]:
data_gazeta_train.head()

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01


Statistics for datasets

In [ ]:
print(f'gazeta: train:{len(data_gazeta_train)} | val:{len(data_gazeta_val)}')
print(f'ria: train:{len(data_ria_train)} | val:{len(data_ria_val)}')

gazeta: train:52285 | val:5265
ria: train:321389 | val:6559


I will use 2 metrics: BLEU and ROUGE. They more correlate with human evaluation.

In [6]:
def calc_scores(references, predictions):
    Metrics = namedtuple("Metrics", "BLEU, ROUGE")
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    Metrics.BLEU = corpus_bleu([[r] for r in references], predictions)
    print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    # rouge = Rouge()
    # scores = rouge.get_scores(predictions, references, avg=True)
    # Metrics.ROUGE = scores
    # print("ROUGE: ", scores)
    return Metrics

### Baseline lead rows

In [8]:
def calc_lead_rows_score(data, n=1, lower=True, summary = 'title'):
    references = []
    predictions = []

    for text, summary in data[['text', summary]].values:
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = text if not lower else text.lower()
        sentences = [sentence for sentence in sent_tokenize(text)] 
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)
    return calc_scores(references, predictions)

In [ ]:
scores_ria = calc_lead_rows_score(data_ria_val, n=1)

Ref: перспективные направления развития транспортной отрасли москвы
Hyp: на встрече с журналистами были обсуждены следующие вопросы:- транспорт и связь москвы ветеранам ко дню победы;- взаимоотношения департамента с федеральной антимонопольной службой.
BLEU:  0.20980822305084199


In [ ]:
scores_gazeta = calc_lead_rows_score(data_gazeta_val, n=3, summary = 'summary')

Ref: в сша заявили о создании коалиции для патрулирования ормузского пролива. она будет составлена из всех стран мира. при этом россия утверждает, что до сих пор не получала приглашения к участию. о создании такого союза начали говорить после ряда инцидентов, произошедших в районе персидского залива с участием ирана. последний имел место 19 июля, когда тегеран задержал британский танкер в своих территориальных водах.
Hyp: сша создают коалицию, чтобы патрулировать ормузский пролив. в нее войдут страны «по всему миру», заявил госсекретарь соединенных штатов америки майк помпео. «мы работаем над тем, чтобы изменить поведение руководства исламской республики иран.
BLEU:  0.4423951640847578


## BART model

**Pipeline**:
1. Download and work out with datasets.
2. Train BPE tokenizer 50000 rows will be enough for training.
3. Split dataset and corrupt it.
4. Choose config that is suitable for these datasets.
5. Save model's weights .
6. Train, then generate using greedy/beam search.
7. Repeat experiement few times for more reliable results.

In [ ]:
MAX_LENGTH = 300

Train and load tokenizer

In [ ]:
CustomTokenizer().train(data_ria_train['text'].values)
tokenizer = CustomTokenizer.load_from_pretrained(MAX_LENGTH)

Number of tokens for each dataset

In [ ]:
for data in [data_ria_train, data_ria_val, data_gazeta_train, data_gazeta_val]:
  stat = tokenizer.encode_batch(data['text'].values)
  len_ = 0
  for i in stat:
    len_ += len(i.ids)
  print(f'num_tokens:{len_}')

num_tokens:104842336
num_tokens:2155983
num_tokens:38909721
num_tokens:3948127


## Pre-training

In [7]:
LR = 1e-4
BATCH_SIZE = 8
EPOCHS = 5
ACC_STEP = 16
ENCODER_LAYERS = 6
DECODER_LAYERS = 6
COL_ARTICLE = 'text'
COL_SUMMARY = 'title'
DATA_VAL_PRETRAIN = data_ria_val
DATA_TRAIN_PRETRAIN = data_ria_train
CHECKPOINT_PATH = 'model/checkpoints/pretrain'

In [ ]:
config = BartConfig(
    vocab_size = tokenizer.get_vocab_size(), 
    pad_token_id = tokenizer.token_to_id("<pad>"),
    bos_token_id = tokenizer.token_to_id("<s>"),
    eos_token_id = tokenizer.token_to_id("</s>"),
    encoder_layers = ENCODER_LAYERS,
    decoder_layers = DECODER_LAYERS
    )

parameters = {
    'lr': LR,
    'batch_size': BATCH_SIZE,
    'acc_step': ACC_STEP,
    'max_length': MAX_LENGTH
}

parameters = Box(parameters)

In [ ]:
model = BART(
    bart_config = config,
    parameters = parameters,
    data_train = DATA_TRAIN_PRETRAIN,
    data_val = DATA_VAL_PRETRAIN,
    col_article = COL_ARTICLE,
    col_summary = COL_SUMMARY,
    tokenizer = tokenizer
    )

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINT_PATH,
    monitor='bleu',
    mode='max'
)

early_stop_callback = EarlyStopping(
    monitor='bleu',
    min_delta=0.00,
    patience=2,
    verbose=False,
    mode='max'
)

trainer = Trainer(
    gpus=1, max_epochs=EPOCHS,
    callbacks = [early_stop_callback, checkpoint_callback]
    )
trainer.fit(model)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /lightning_logs

In [ ]:
torch.cuda.empty_cache()

## Fine-tuning

In [7]:
DATA_TRAIN_FINETUNE = data_ria_train.iloc[:100_000]
DATA_VAL_FINETUNE = data_ria_val.iloc[:100_000]
ENCODER_LAYERS = 6
DECODER_LAYERS = 6
# PRETRAINED_PATH = os.path.join(CHECKPOINT_PATH, 'best-model.ckpt')
SAVE_PATH = 'model/checkpoints/finetune'
COL_ARTICLE = 'text'
COL_SUMMARY = 'title'
MAX_LENGTH = 750
LR = 1e-5
MAX_LR = 5e-5
PCT_START = 0.06
EPOCHS = 5
BATCH_SIZE = 8
ACC_STEP = 8

In [8]:
tokenizer = CustomTokenizer.load_from_pretrained(MAX_LENGTH)

In [9]:
config = BartConfig(
    vocab_size = tokenizer.get_vocab_size(), 
    pad_token_id = tokenizer.token_to_id("<pad>"),
    bos_token_id = tokenizer.token_to_id("<s>"),
    eos_token_id = tokenizer.token_to_id("</s>"),
    encoder_layers = ENCODER_LAYERS,
    decoder_layers = DECODER_LAYERS
    )

parameters = {
    'lr': LR,
    'max_lr': MAX_LR,
    'pct_start': PCT_START,
    'num_epoch': EPOCHS,
    'batch_size': BATCH_SIZE,
    'acc_step': ACC_STEP,
    'max_length': MAX_LENGTH
}

parameters = Box(parameters)

In [10]:
model = BART_finetune(
    bart_config = config,
    parameters = parameters,
    data_train = DATA_TRAIN_FINETUNE,
    data_val = DATA_VAL_FINETUNE,
    col_article = COL_ARTICLE,
    col_summary = COL_SUMMARY,
    tokenizer = tokenizer
)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath=SAVE_PATH,
    monitor='bleu',
    mode='max'
)

early_stop_callback = EarlyStopping(
    monitor='bleu',
    min_delta=0.00,
    patience=2,
    verbose=False,
    mode='max'
)

trainer = Trainer(
    gpus=1, max_epochs=EPOCHS,
    callbacks = [early_stop_callback, checkpoint_callback]
    )
trainer.fit(model)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/ 

In [ ]:
torch.cuda.empty_cache()

## Examples

In [11]:
PATH = os.path.join(SAVE_PATH, 'epoch=3-step=49999.ckpt')

In [ ]:
bart = load_bart(config, PATH)
tokenizer = CustomTokenizer.load_from_pretrained()
test_loader = FineTuneLoader.load(
    data_ria_val,
    tokenizer,
    'text',
    'title'
)

bart.cuda()
bart.eval()

In [19]:
ref = []
pred = []

with torch.no_grad():
  for i, args in enumerate(tqdm(test_loader)):
    generated = bart.generate(args['input_ids'].cuda())
    generated = generated.cpu().numpy().tolist()
    decoder_inputs = args['decoder_input_ids'].numpy().tolist()

    pred.extend(tokenizer.decode_batch(generated))
    ref.extend(tokenizer.decode_batch(decoder_inputs))
    if i == 100: break
calc_scores(ref, pred)

  8%|▊         | 100/1284 [00:32<06:26,  3.07it/s]


Ref: <s>сотрудников google в милане осудили за размещение некорректного ролика</s><pad><pad><pad><pad><pad><pad>
Hyp: <mask>в сша приговор приговор осужденным к 6 месяцам тюрьмы</s><pad><pad><pad><pad><pad><pad>
BLEU:  0.4614935695749583


__main__.Metrics